## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,49.93,94,68,5.95,broken clouds
1,1,Mar Del Plata,AR,-38.0023,-57.5575,55.09,98,100,1.99,overcast clouds
2,2,Rio Grande,BR,-32.0350,-52.0986,62.71,97,51,8.48,broken clouds
3,3,Chokurdakh,RU,70.6333,147.9167,55.87,42,47,10.78,scattered clouds
4,4,Sinnamary,GF,5.3833,-52.9500,79.07,81,22,5.53,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
4,4,Sinnamary,GF,5.3833,-52.9500,79.07,81,22,5.53,few clouds
8,8,Puerto Ayora,EC,-0.7393,-90.3518,70.56,92,100,3.00,overcast clouds
9,9,Hithadhoo,MV,-0.6000,73.0833,77.43,89,100,16.69,heavy intensity rain
10,10,Torbay,CA,47.6666,-52.7314,70.57,88,100,17.27,overcast clouds
11,11,Westport,US,41.1415,-73.3579,77.94,68,0,6.91,clear sky
16,16,Jamestown,US,42.0970,-79.2353,71.24,40,0,8.05,clear sky
17,17,Albany,US,42.6001,-73.9662,74.05,64,61,5.08,broken clouds
18,18,Huangchuan,CN,32.1272,115.0394,80.67,87,6,5.12,clear sky
19,19,Jibuti,DJ,11.5000,42.5000,82.06,50,94,8.72,light rain
25,25,Whitehorse,CA,60.7161,-135.0538,77.77,27,75,10.36,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                321
City                   321
Country                321
Lat                    321
Lng                    321
Max Temp               321
Humidity               321
Cloudiness             321
Wind Speed             321
Weather Description    321
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
4,4,Sinnamary,GF,5.3833,-52.9500,79.07,81,22,5.53,few clouds
8,8,Puerto Ayora,EC,-0.7393,-90.3518,70.56,92,100,3.00,overcast clouds
9,9,Hithadhoo,MV,-0.6000,73.0833,77.43,89,100,16.69,heavy intensity rain
10,10,Torbay,CA,47.6666,-52.7314,70.57,88,100,17.27,overcast clouds
11,11,Westport,US,41.1415,-73.3579,77.94,68,0,6.91,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Sinnamary,GF,79.07,few clouds,5.3833,-52.9500,
8,Puerto Ayora,EC,70.56,overcast clouds,-0.7393,-90.3518,
9,Hithadhoo,MV,77.43,heavy intensity rain,-0.6000,73.0833,
10,Torbay,CA,70.57,overcast clouds,47.6666,-52.7314,
11,Westport,US,77.94,clear sky,41.1415,-73.3579,
16,Jamestown,US,71.24,clear sky,42.0970,-79.2353,
17,Albany,US,74.05,broken clouds,42.6001,-73.9662,
18,Huangchuan,CN,80.67,clear sky,32.1272,115.0394,
19,Jibuti,DJ,82.06,light rain,11.5000,42.5000,
25,Whitehorse,CA,77.77,broken clouds,60.7161,-135.0538,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Sinnamary,GF,79.07,few clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
8,Puerto Ayora,EC,70.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Hithadhoo,MV,77.43,heavy intensity rain,-0.6000,73.0833,Scoop Guest House
10,Torbay,CA,70.57,overcast clouds,47.6666,-52.7314,Gallows Cove
11,Westport,US,77.94,clear sky,41.1415,-73.3579,The Westport Inn
...,...,...,...,...,...,...,...
684,Requena,ES,72.79,broken clouds,39.4883,-1.1004,Hotel Avenida
689,Sassandra,CI,74.46,light rain,4.9500,-6.0833,la terrasse
690,Chernyshevskiy,RU,78.94,clear sky,63.0128,112.4714,Chernyseyskiy
691,Ucluelet,CA,77.31,few clouds,48.9329,-125.5528,Black Rock Oceanfront Resort


In [10]:
hotel_df.count()

City                   321
Country                321
Max Temp               321
Weather Description    321
Lat                    321
Lng                    321
Hotel Name             321
dtype: int64

In [29]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0, how="any")


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
clean_hotel_df


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Sinnamary,GF,79.07,few clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
8,Puerto Ayora,EC,70.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Hithadhoo,MV,77.43,heavy intensity rain,-0.6000,73.0833,Scoop Guest House
10,Torbay,CA,70.57,overcast clouds,47.6666,-52.7314,Gallows Cove
11,Westport,US,77.94,clear sky,41.1415,-73.3579,The Westport Inn
...,...,...,...,...,...,...,...
684,Requena,ES,72.79,broken clouds,39.4883,-1.1004,Hotel Avenida
689,Sassandra,CI,74.46,light rain,4.9500,-6.0833,la terrasse
690,Chernyshevskiy,RU,78.94,clear sky,63.0128,112.4714,Chernyseyskiy
691,Ucluelet,CA,77.31,few clouds,48.9329,-125.5528,Black Rock Oceanfront Resort


In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>County</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""



In [33]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [47]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(40.0, 41.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))